In [1]:
# !pip3 install imbalanced-learn

### Set up Synthetic Imbalanced Data

In [2]:
# check version number
import imblearn
import sklearn
from sklearn.datasets import make_classification

print(imblearn.__version__)
import pandas as pd
data = pd.read_csv('credit_split_2.csv')

0.8.1


In [3]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,84692.0,1.125592,-0.147999,1.151008,1.158851,-0.823914,0.298829,-0.708732,0.294683,0.750838,...,-0.037453,0.024388,-0.052720,-0.003615,0.405682,-0.404597,0.063357,0.024091,12.99,0
1,84693.0,-0.439952,0.683758,1.225814,0.639113,0.716765,0.089295,0.657718,0.034213,-0.472679,...,0.127443,0.588590,-0.145728,-0.322900,-0.299744,-0.310295,0.248317,-0.011493,19.00,0
2,84694.0,-4.868108,1.264420,-5.167885,3.193648,-3.045621,-2.096166,-6.445610,2.422536,-3.214055,...,1.269205,0.057657,0.629307,-0.168432,0.443744,0.276539,1.441274,-0.127944,12.31,1
3,84695.0,1.013114,-0.334412,1.305208,0.837406,-1.126833,-0.064321,-0.594753,0.147737,0.536360,...,0.017079,0.112210,-0.016084,0.595033,0.201073,0.278215,0.007457,0.030762,66.60,0
4,84695.0,0.969231,-0.233554,0.238473,0.145793,-0.545741,-0.970680,0.347393,-0.209522,-0.342571,...,-0.362820,-1.417272,0.162136,0.541628,-0.079465,0.268702,-0.101237,0.028234,141.00,0


In [4]:
from sklearn.model_selection import train_test_split


X = data.drop('Class', axis=1)
y = data['Class']

seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

from collections import Counter

# summarize class distribution
counter = Counter(y_train)
print(counter)
counter = Counter(y_test)
print(counter)

Counter({0: 95261, 1: 149})
Counter({0: 46920, 1: 74})


## Test without SMOTE

In [5]:
import xgboost
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_fscore_support, auc, confusion_matrix, classification_report


xgbmod = XGBClassifier()
xgbmod.fit(X_train, y_train)
y_pred = xgbmod.predict(X_test)
predictions = [round(value) for value in y_pred]
precision, recall, fbeta_score, support = precision_recall_fscore_support(y_test, predictions)
result = auc(recall, precision)
print("AUC: %.2f%%" % (result))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:42:49] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
AUC: 0.17%
[[46915     5]
 [   13    61]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     46920
           1       0.92      0.82      0.87        74

    accuracy                           1.00     46994
   macro avg       0.96      0.91      0.94     46994
weighted avg       1.00      1.00      1.00     46994



### Oversample to be Equal Distribution

In [7]:
from imblearn.over_sampling import SMOTE

# transform the dataset
oversample = SMOTE(sampling_strategy= 0.5, n_jobs=-1)
X_train_over, y_train_over = oversample.fit_resample(X_train, y_train)

In [8]:
# summarize the new class distribution
counter = Counter(y_train_over)
print(counter)

Counter({0: 95261, 1: 47630})


In [9]:
xgbmod = XGBClassifier()
xgbmod.fit(X_train_over, y_train_over)
y_pred = xgbmod.predict(X_test)
predictions = [round(value) for value in y_pred]
precision, recall, fbeta_score, support = precision_recall_fscore_support(y_test, predictions)
result = auc(recall, precision)
print("AUC: %.2f%%" % (result))
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

/usr/local/lib/python3.8/dist-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:43:00] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
AUC: 0.12%
[[46903    17]
 [   10    64]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     46920
           1       0.79      0.86      0.83        74

    accuracy                           1.00     46994
   macro avg       0.89      0.93      0.91     46994
weighted avg       1.00      1.00      1.00     46994

